In [ ]:
%matplotlib inline

## Importation des biblio

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.core.display import HTML

## Importation de la data

In [ ]:
df=pd.read_csv('../input/pyms-diabete/diabete.csv')
df.shape

In [ ]:
display(HTML(df.head(768).to_html()))

In [ ]:
df.columns

## Splitting

In [ ]:
X = df.drop(['diabete'], axis=1)
y = df.diabete

## Analyse data 'entrée'

D'apres mes connaissance médicale, je sais bien que 'insuline ,'glucose' sont dépendant alors je vais droper glucose qui est depend de l'insuline.

In [ ]:
X=X.drop(['glucose'],axis=1)

In [ ]:
X.count()

In [ ]:
X.describe()

## Missing data

In [ ]:
X.isnull().values.any()

Jusqu'à ici tous est bon, On a pas des NaN à dire. 
Compte tenu de la description de la dataset et la nature de ces éléments, on a trouvé qu'il y'a des NaN dissimulé en des 0. Alors, il faut tous d'abord définir les colonnes concernées.

In [ ]:
missing_columns=(X==0).sum()
print(missing_columns)

En testant, on sait que 'n_pregnant', 'glucose', 'tension', 'thickness', 'insulin', 'bmi','diabete' contiennet des 0.
Logiquement parlant, si un sujet a 0 en : 'glucose', 'tension', 'thickness', 'insulin', 'bmi',diabete c'est à dire que c'est un sujet mort. je vais donc remplacer les 0 par des NaN et puis par la moyenne.
Mais avant de cela.

In [ ]:
from numpy import nan
X[['tension','thickness','insulin','bmi']] = X[['tension','thickness','insulin','bmi']].replace(0, nan)

In [ ]:
X.isnull().values.any()

In [ ]:
X[np.isnan(df[['tension','thickness','insulin','bmi']])]

In [ ]:
#X.fillna(df.mean(), inplace=True)
X=X.fillna(value={'tension': X.tension.mean(),'thickness': X.thickness.mean(),'insulin': X.insulin.mean(),'bmi': X.bmi.mean()})

In [ ]:
X.isnull().values.any()

In [ ]:
X.columns

In [ ]:
X.hist(bins=80)

In [ ]:
'''def replace_0(df,col) :
    df1 = df.copy()
    n = df.shape[0]
    m = df[col].mean()
    s = df[col].std()
    for i in range(n) :
        if df.loc[i,col]==0 :
            df1.loc[i,col] = np.random.normal(m,s)
    return df1'''

'Le but de cette fonction est de remplacer les 0 par des mean directement mais moi j ai travailler autrement'

## Standarisation 

In [ ]:
from sklearn import preprocessing
scaler=preprocessing.StandardScaler()
X=scaler.fit_transform(X)

In [ ]:
sns.pairplot(df)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

## Construction de modele

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_lr=lr.predict(X_test)
print(accuracy_score(y_test,y_lr))
print(confusion_matrix(y_test,y_lr))
print(classification_report(y_test,y_lr))

In [ ]:
probas=lr.predict_proba(X_test)
print(probas)

In [ ]:
df_probas=pd.DataFrame(probas,columns=['proba_ND','proba_D'])
df_probas['y']=np.array(y_test)
df_probas.head(100)

In [ ]:
sns.kdeplot(df_probas.proba_D[df_probas.y==1],color='red')
sns.kdeplot(df_probas.proba_D[df_probas.y==0],color='blue')
sns.kdeplot(1-df_probas.proba_ND[df_probas.y==0],color='green')

### Randon Forest

In [ ]:
from sklearn import ensemble
rf=ensemble.RandomForestClassifier()
rf.fit(X_train,y_train)
y_rf=rf.predict(X_test)
print(accuracy_score(y_test,y_rf))
print(confusion_matrix(y_test,y_rf))
print(classification_report(y_test,y_rf))

In [ ]:
rfa = ensemble.RandomForestClassifier(n_estimators=20, min_samples_leaf=15, max_features=5)
rfa.fit(X_train, y_train)
y_rfa = rfa.predict(X_test)
print(accuracy_score(y_test, y_rfa))
print(confusion_matrix(y_test,y_rfa))
print(classification_report(y_test,y_rfa))

In [ ]:
from sklearn.model_selection import validation_curve
params = np.arange(1, 300,step=30)
train_score, val_score = validation_curve(rf, X, y, 'n_estimators', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('n_estimators')
plt.ylabel('score');

In [ ]:
from sklearn import model_selection

In [ ]:
param_grid = {'n_estimators': [1,100,500],'min_samples_leaf': [1,10,50]}
estimator = ensemble.RandomForestClassifier()
rf_gs = model_selection.GridSearchCV(estimator, param_grid)

In [ ]:
rf_gs.fit(X_train, y_train)
print(rf_gs.best_params_)

In [ ]:
rfb = rf_gs.best_estimator_
y_rfb = rfb.predict(X_test)
print(accuracy_score(y_test, y_rfb))
print(confusion_matrix(y_test, y_rfb))
print(classification_report(y_test, y_rfb))

In [ ]:
importances = rfb.feature_importances_
indices = np.argsort(importances)
plt.figure(figsize=(15,8))
plt.barh(range(len(indices)),importances[indices],color='b',align='center')
plt.yticks(range(len(indices)),df.columns[indices])
plt.title('Importances des caractéristiques')

### XGB

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBClassifier()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
print(accuracy_score(y_test, y_xgb))
print(confusion_matrix(y_test, y_xgb))
print(classification_report(y_test, y_xgb))

### NNC

In [ ]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
import numpy as np
nc = NearestCentroid()
nc.fit(X_train, y_train)
y_nc = nc.predict(X_test)
print(accuracy_score(y_test, y_nc))
print(confusion_matrix(y_test, y_nc))
print(classification_report(y_test, y_nc))

### SVM

In [ ]:
from sklearn import svm
svm = svm.SVC()
svm.fit(X_train, y_train)
y_svm = svm.predict(X_test)
print(accuracy_score(y_test, y_svm))
print(confusion_matrix(y_test, y_svm))
print(classification_report(y_test, y_svm))